In [1]:
import os
import argparse
import logging
import pickle
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity

# Try importing faiss for efficient similarity search.
try:
    import faiss
except ImportError:
    faiss = None
    logging.info("Faiss is not installed; falling back to cosine similarity.")

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class LegalDocumentFinder:
    def __init__(self, model_name: str = "vinai/phobert-base", device: str = None):
        # Set device to CUDA if available, unless explicitly specified.
        self.device = torch.device(device) if device else torch.device("cuda" if torch.cuda.is_available() else "cpu")
        logger.info("Using device: %s", self.device)
        
        # Load tokenizer and model.
        self.tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
        self.model = AutoModel.from_pretrained(model_name)
        self.model.to(self.device)
        self.model.eval()

        self.embeddings = None  # Numpy array containing embeddings.
        self.df = None          # DataFrame with the legal documents.
        self.index = None       # FAISS index for efficient similarity search (optional).

    @staticmethod
    def load_data(file_path: str) -> pd.DataFrame:
        """Load the CSV file with legal texts, handling common encoding issues."""
        try:
            df = pd.read_csv(file_path, encoding='utf-8-sig', sep=',', quotechar='"')
            logger.info("Loaded data using utf-8-sig encoding.")
        except Exception as e:
            logger.warning("utf-8-sig encoding failed: %s. Trying utf-16...", e)
            df = pd.read_csv(file_path, encoding='utf-16', sep=',', quotechar='"')
        # Clean DataFrame: remove rows with missing text or duplicates.
        df = df.dropna(subset=["truncated_text", "dieu"]).drop_duplicates(subset=["truncated_text"])
        df.reset_index(drop=True, inplace=True)
        logger.info("Data cleaned: %d rows.", len(df))
        return df

    def batch_encode_texts(self, texts: list, batch_size: int = 32, max_length: int = 256) -> np.ndarray:
        """Encode texts in batches using GPU if available."""
        embeddings = []
        logger.info("Starting batch encoding for %d texts...", len(texts))
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding Batches"):
            batch_texts = texts[i: i + batch_size]
            inputs = self.tokenizer(batch_texts, return_tensors="pt", truncation=True,
                                      max_length=max_length, padding="max_length")
            inputs = {k: v.to(self.device) for k, v in inputs.items()}
            with torch.no_grad():
                outputs = self.model(**inputs)
            # Extract the [CLS] token embedding from the last hidden state.
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(batch_embeddings)
        embeddings = np.concatenate(embeddings, axis=0)
        logger.info("Completed encoding. Embeddings shape: %s", embeddings.shape)
        return embeddings

    def cache_embeddings(self, cache_path: str):
        """Save computed embeddings and DataFrame to disk."""
        with open(cache_path, "wb") as f:
            pickle.dump({"df": self.df, "embeddings": self.embeddings}, f)
        logger.info("Embeddings cached to %s.", cache_path)

    def load_cached_embeddings(self, cache_path: str) -> bool:
        """Load cached embeddings and DataFrame from disk if available."""
        if os.path.exists(cache_path):
            with open(cache_path, "rb") as f:
                cache = pickle.load(f)
                self.df = cache["df"]
                self.embeddings = cache["embeddings"]
            logger.info("Loaded cached embeddings from %s.", cache_path)
            return True
        return False

    def build_index(self):
        """Build an index for efficient similarity search using FAISS, if available."""
        if faiss is not None and self.embeddings is not None:
            d = self.embeddings.shape[1]
            self.index = faiss.IndexFlatL2(d)
            # Normalize the embeddings for cosine similarity approximation.
            faiss.normalize_L2(self.embeddings)
            self.index.add(self.embeddings.astype(np.float32))
            logger.info("FAISS index built with %d vectors.", self.index.ntotal)
        else:
            logger.info("FAISS not available or embeddings not computed; skipping index build.")

    def encode_text(self, text: str, max_length: int = 256) -> np.ndarray:
        """Encode a single input text into its embedding."""
        inputs = self.tokenizer(text, return_tensors="pt", truncation=True,
                                max_length=max_length, padding="max_length")
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        with torch.no_grad():
            outputs = self.model(**inputs)
        return outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()

    def find_similar_articles(self, input_text: str, top_k: int = 5) -> pd.DataFrame:
        """Retrieve the top_k similar legal documents for a given input text."""
        input_embedding = self.encode_text(input_text).reshape(1, -1)
        
        # Normalize the input embedding.
        input_norm = input_embedding / np.linalg.norm(input_embedding, axis=1, keepdims=True)
        
        if self.index is not None:
            # Use FAISS index for similarity search.
            input_embedding_norm = input_norm.astype(np.float32)
            distances, indices = self.index.search(input_embedding_norm, top_k)
            # Approximate cosine similarity from Euclidean distances.
            similarities = 1 - distances.flatten() / 2  
            top_indices = indices.flatten()
            logger.info("Similar articles found using FAISS.")
        else:
            # Fall back to cosine similarity calculation.
            matrix = np.stack(self.df["embedding"].values)
            similarities = cosine_similarity(input_embedding, matrix).flatten()
            top_indices = similarities.argsort()[::-1][:top_k]
            similarities = similarities[top_indices]
            logger.info("Similar articles found using cosine similarity.")
        
        results = self.df.iloc[top_indices][["dieu", "truncated_text"]].copy()
        results["similarity"] = similarities
        return results

def parse_args():
    parser = argparse.ArgumentParser(description="Automated System for Identifying Vietnamese Legal Documents")
    parser.add_argument("--data_file", type=str, default="sent_truncated_vbpl_legal_only.csv",
                        help="CSV file path containing legal texts.")
    parser.add_argument("--cache_file", type=str, default="embeddings_cache.pkl",
                        help="Path to save/load embeddings cache.")
    parser.add_argument("--batch_size", type=int, default=32, help="Batch size for text encoding.")
    parser.add_argument("--use_faiss", action="store_true", help="Build FAISS index for similarity search (if installed).")
    # Use parse_known_args to handle extra parameters injected by the Jupyter kernel.
    args, unknown = parser.parse_known_args()
    if unknown:
        logger.debug("Ignoring unknown arguments: %s", unknown)
    return args

def main():
    args = parse_args()

    # Initialize the legal document finder.
    finder = LegalDocumentFinder()

    # Load and preprocess data.
    finder.df = finder.load_data(args.data_file)

    # Check for cached embeddings.
    if not finder.load_cached_embeddings(args.cache_file):
        all_texts = finder.df["truncated_text"].tolist()
        finder.embeddings = finder.batch_encode_texts(all_texts, batch_size=args.batch_size)
        # Store embeddings in the DataFrame for later use.
        finder.df["embedding"] = list(finder.embeddings)
        finder.cache_embeddings(args.cache_file)
    
    # Build FAISS index if the flag is set.
    if args.use_faiss:
        finder.build_index()
    
    # Demo search.
    demo_text = "Biên bản họp Hội đồng thành viên phải được giữ nguyên và không được phép thay đổi"
    results = finder.find_similar_articles(demo_text)
    
    # Output the results.
    print("\n🔍 Top Related Articles:")
    for idx, row in results.iterrows():
        print(f"\n📘 {row['dieu']} (Similarity: {row['similarity']:.2f})")
        print(f"📝 {row['truncated_text']}")

if __name__ == '__main__':
    main()


Encoding Batches: 100%|███████████████████████████████████████████████████████████████| 290/290 [00:34<00:00,  8.42it/s]



🔍 Top Related Articles:

📘 Điều 179 (Similarity: 0.68)
📝 Hợp đồng gia công phải được lập thành văn bản hoặc bằng hình thức khác có giá trị pháp lý tương đương.

📘 Điều 251 (Similarity: 0.67)
📝 Hợp đồng dịch vụ quá cảnh phải được lập thành văn bản hoặc bằng hình thức khác có giá trị pháp lý tương đương.

📘 Điều 159 (Similarity: 0.67)
📝 Hợp đồng uỷ thác mua bán hàng hoá phải được lập thành văn bản hoặc bằng hình thức khác có giá trị pháp lý tương đương.

📘 Điều 168 (Similarity: 0.66)
📝 Hợp đồng đại lý phải được lập thành văn bản hoặc bằng hình thức khác có giá trị pháp lý tương đương.

📘 Điều 142 (Similarity: 0.65)
📝 Hợp đồng đại diện cho thương nhân phải được lập thành văn bản hoặc bằng hình thức khác có giá trị pháp lý tương đương.


In [3]:
def main():
    args = parse_args()

    # Initialize the legal document finder.
    finder = LegalDocumentFinder()

    # Load and preprocess data.
    finder.df = finder.load_data(args.data_file)

    # Check for cached embeddings.
    if not finder.load_cached_embeddings(args.cache_file):
        all_texts = finder.df["truncated_text"].tolist()
        finder.embeddings = finder.batch_encode_texts(all_texts, batch_size=args.batch_size)
        # Store embeddings in the DataFrame for later use.
        finder.df["embedding"] = list(finder.embeddings)
        finder.cache_embeddings(args.cache_file)
    
    # Build FAISS index if the flag is set.
    if args.use_faiss:
        finder.build_index()
    
    # Demo search.
    demo_text = "không được phép thay đổi"
    results = finder.find_similar_articles(demo_text)
    
    # Output the results.
    print("\n🔍 Top Related Articles:")
    for idx, row in results.iterrows():
        print(f"\n📘 {row['dieu']} (Similarity: {row['similarity']:.2f})")
        print(f"📝 {row['truncated_text']}")

if __name__ == '__main__':
    main()


🔍 Top Related Articles:

📘 Điều 392 (Similarity: 0.56)
📝 Sửa đổi đề nghị do bên được đề nghị đề xuất Khi bên được đề nghị đã chấp nhận giao kết hợp đồng nhưng có nêu điều kiện hoặc sửa đổi đề nghị thì coi như người này đã đưa ra đề nghị mới.

📘 Điều 12 (Similarity: 0.56)
📝 Trừ trường hợp có thoả thuận khác, các bên được coi là mặc nhiên áp dụng thói quen trong hoạt động thương mại đã được thiết lập giữa các bên đó mà các bên đã biết hoặc phải biết nhưng không được trái với quy định của pháp luật.

📘 Điều 116 (Similarity: 0.55)
📝 Người tham gia thủ tục phá sản là người nước ngoài Người tham gia thủ tục phá sản là người nước ngoài phải thực hiện theo quy định của pháp luật về phá sản của Việt Nam.

📘 Điều 644 (Similarity: 0.54)
📝 Việc thừa kế của những người có quyền thừa kế di sản của nhau mà chết trong cùng một thời điểm Trong trường hợp những người có quyền thừa kế di sản của nhau đều chết trong cùng một thời điểm hoặc được coi là chết trong cùng một thời điểm do không thể xác định đ